In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler, KMeansSMOTE, SVMSMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# kagglehub.dataset_download("joshmcadams/oranges-vs-grapefruit")
citrus = pd.read_csv("E:\Workings\Thesis\Percentile v SMOTE\Datasets smote\citrus.csv")

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
C:\Users\danis\AppData\Local\Temp\ipykernel_9272\1082590174.py:2: SyntaxWarning: invalid escape sequence '\W'
  citrus = pd.read_csv("E:\Workings\Thesis\Percentile v SMOTE\Datasets smote\citrus.csv")


In [3]:
citrus

,name,diameter,weight,red,green,blue
0,orange,2.96,86.76,172,85,2
1,orange,3.91,88.05,166,78,3
2,orange,4.42,95.17,156,81,2
3,orange,4.47,95.60,163,81,4
4,orange,4.48,95.76,161,72,9
...,...,...,...,...,...,...
9995,grapefruit,15.35,253.89,149,77,20
9996,grapefruit,15.41,254.67,148,68,7
9997,grapefruit,15.59,256.50,168,82,20
9998,grapefruit,15.92,260.14,142,72,11


In [4]:
citrus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      10000 non-null  object 
 1   diameter  10000 non-null  float64
 2   weight    10000 non-null  float64
 3   red       10000 non-null  int64  
 4   green     10000 non-null  int64  
 5   blue      10000 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 468.9+ KB


In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
label_encoder = LabelEncoder()
citrus['name'] = label_encoder.fit_transform(citrus['name'])

In [7]:
citrus['name'].value_counts()

name
1    5000
0    5000
Name: count, dtype: int64

In [8]:
from imblearn.datasets import make_imbalance

In [9]:
from sklearn.model_selection import train_test_split
from collections import Counter

In [10]:
df_resampled, y_resampled = make_imbalance(citrus, citrus['name'], sampling_strategy={ 0 : 1700, 1: 5000},random_state=42)
df_resampled2 = df_resampled.copy()     # saving same data to be used for our proposed method 
print("Original class distribution:", Counter(y_resampled))
del df_resampled['name']

Original class distribution: Counter({1: 5000, 0: 1700})


In [11]:
# Function to calculate the required metrics 
def get_metrics(y_true, y_pred, y_prob):
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None)
    auc = roc_auc_score(y_true, y_prob[:, 1])
    return accuracy, precision, recall, f1, auc

# Define classifiers
classifiers = {
    'GaussianNB': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'RandomForest': RandomForestClassifier(random_state=42)
}

# Function to classify and store metrics
def classify_and_store(classifiers, X_train, y_train, X_test, y_test, oversampler=None):
    if oversampler:
        # Apply oversampling
        X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
    else:
        X_train_resampled, y_train_resampled = X_train, y_train
    
    # Store the metrics for each classifier
    classifier_metrics = {}
    
    for name, clf in classifiers.items():
        # Train the classifier
        clf.fit(X_train_resampled, y_train_resampled)
        
        # Predict on the test set
        y_pred = clf.predict(X_test)
        y_prob = clf.predict_proba(X_test)
        
        # Get metrics
        accuracy, precision, recall, f1, auc = get_metrics(y_test, y_pred, y_prob)
        classifier_metrics[name] = {
            'accuracy': accuracy,
            'precision_0': precision[0],
            'precision_1': precision[1],
            'recall_0': recall[0],
            'recall_1': recall[1],
            'f1_0': f1[0],
            'f1_1': f1[1],
            'auc_0': auc if len(np.unique(y_test)) == 2 else None,
            'auc_1': auc if len(np.unique(y_test)) == 2 else None
        }
    
    return classifier_metrics


# Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(df_resampled, y_resampled, test_size=0.2, random_state=42)

# Standardizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 1: Perform baseline evaluation (without oversampling) and store results
oversamplers = {
    'RandomOverSampler': RandomOverSampler(random_state=42, sampling_strategy='minority'),
    'SMOTE': SMOTE(random_state=42, sampling_strategy='minority'),
    'ADASYN': ADASYN(random_state=42, sampling_strategy='minority'),
    'KMeansSMOTE': KMeansSMOTE(random_state=42, sampling_strategy='minority'),
    'SVMSMOTE': SVMSMOTE(random_state=42, sampling_strategy='minority')
}

# Store all results in a single DataFrame
all_results = []

# Step 2: Baseline (no oversampling)
baseline_metrics = classify_and_store(classifiers, X_train, y_train, X_test, y_test)
for clf_name, m in baseline_metrics.items():
    all_results.append({
        'Classifier': clf_name,
        'Oversampler': 'None',
        'Accuracy': m['accuracy'],
        'Precision(0)': m['precision_0'],
        'Precision(1)': m['precision_1'],
        'Recall (0)': m['recall_0'],
        'Recall (1)': m['recall_1'],
        'F1(0)': m['f1_0'],
        'F1(1)': m['f1_1'],
        'AUC(0)': m['auc_0'],
        'AUC(1)': m['auc_1']
    })

# Step 3: Loop over classifiers and then oversamplers
for clf_name, clf in classifiers.items():
    for sampler_name, oversampler in oversamplers.items():
        oversampled_metrics = classify_and_store({clf_name: clf}, X_train, y_train, X_test, y_test, oversampler)
        
        # Now m will be the dictionary with metrics for the classifier
        m = oversampled_metrics[clf_name]
        
        all_results.append({
            'Classifier': clf_name,
            'Oversampler': sampler_name,
            'Accuracy': m['accuracy'],
            'Precision(0)': m['precision_0'],
            'Precision(1)': m['precision_1'],
            'Recall (0)': m['recall_0'],
            'Recall (1)': m['recall_1'],
            'F1(0)': m['f1_0'],
            'F1(1)': m['f1_1'],
            'AUC(0)': m['auc_0'],
            'AUC(1)': m['auc_1']
        })

# Convert to DataFrame
results_df = pd.DataFrame(all_results)


_______________________________________________________________________________________________________________________________________________________________________

In [12]:
citrus[citrus['name']==0].corr().abs().sum().sort_values()

name        0.000000
green       1.017065
blue        1.017437
red         1.022842
diameter    2.015328
weight      2.015351
dtype: float64

In [13]:
df_resampled2[df_resampled2['name']==0].count()

name        1700
diameter    1700
weight      1700
red         1700
green       1700
blue        1700
dtype: int64

In [14]:
4019-1700

2319

In [15]:
minority_samples = df_resampled2[df_resampled2['name']==0]
random_values = minority_samples['weight'].sample(n=2319, replace=True)
random_values = random_values.reset_index()
random_values = pd.DataFrame(random_values)
del random_values['index']
random_values

,weight
0,186.08
1,190.54
2,189.44
3,191.85
4,198.99
...,...
2314,201.87
2315,213.20
2316,179.26
2317,177.01


In [16]:
random_values['diameter'] = np.nan
random_values['red'] = np.nan
random_values['green'] = np.nan
random_values['blue'] = np.nan
random_values['name'] = 0

In [17]:
random_values

,weight,diameter,red,green,blue,name
0,186.08,NaN,NaN,NaN,NaN,0
1,190.54,NaN,NaN,NaN,NaN,0
2,189.44,NaN,NaN,NaN,NaN,0
3,191.85,NaN,NaN,NaN,NaN,0
4,198.99,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...
2314,201.87,NaN,NaN,NaN,NaN,0
2315,213.20,NaN,NaN,NaN,NaN,0
2316,179.26,NaN,NaN,NaN,NaN,0
2317,177.01,NaN,NaN,NaN,NaN,0


In [18]:
def calculate_percentiles(nums):
    indexed_nums = [(num, i) for i, num in enumerate(nums)]
    sorted_nums = []
    for num_index in indexed_nums:
        inserted = False
        for i, sorted_num_index in enumerate(sorted_nums):
            if num_index[0] < sorted_num_index[0]:
                sorted_nums.insert(i, num_index)
                inserted = True
                break
        if not inserted:
            sorted_nums.append(num_index)
    length = len(sorted_nums)
    percentiles = [0] * length
    for i, num_index in enumerate(sorted_nums):
        original_index = num_index[1]
        percentile = ((i + 1) / length) * 100
        percentiles[original_index] = percentile
    return percentiles

def dataframe_to_percentiles(df):
    df_percentiles = df.copy()
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            df_percentiles[column] = calculate_percentiles(df[column].tolist())
    return df_percentiles


In [19]:
random_values['weight'] = calculate_percentiles(random_values['weight'])
random_values['weight'] = 'P' + random_values['weight'].round().astype(str,errors='ignore')

In [20]:
random_values['weight'].unique()

array(['P30.0', 'P38.0', 'P35.0', 'P41.0', 'P55.0', 'P99.0', 'P57.0',
       'P96.0', 'P76.0', 'P47.0', 'P71.0', 'P22.0', 'P37.0', 'P68.0',
       'P64.0', 'P59.0', 'P26.0', 'P79.0', 'P14.0', 'P32.0', 'P86.0',
       'P90.0', 'P1.0', 'P85.0', 'P16.0', 'P72.0', 'P40.0', 'P18.0',
       'P51.0', 'P21.0', 'P63.0', 'P81.0', 'P74.0', 'P89.0', 'P50.0',
       'P48.0', 'P8.0', 'P49.0', 'P29.0', 'P70.0', 'P73.0', 'P77.0',
       'P60.0', 'P80.0', 'P17.0', 'P75.0', 'P54.0', 'P61.0', 'P7.0',
       'P67.0', 'P0.0', 'P44.0', 'P62.0', 'P5.0', 'P69.0', 'P91.0',
       'P95.0', 'P3.0', 'P31.0', 'P20.0', 'P65.0', 'P34.0', 'P36.0',
       'P10.0', 'P27.0', 'P92.0', 'P78.0', 'P43.0', 'P19.0', 'P46.0',
       'P4.0', 'P28.0', 'P88.0', 'P84.0', 'P98.0', 'P82.0', 'P56.0',
       'P33.0', 'P58.0', 'P2.0', 'P9.0', 'P39.0', 'P42.0', 'P23.0',
       'P25.0', 'P12.0', 'P52.0', 'P45.0', 'P13.0', 'P6.0', 'P24.0',
       'P93.0', 'P66.0', 'P100.0', 'P83.0', 'P11.0', 'P53.0', 'P87.0',
       'P15.0', 'P97.0', 'P94

In [21]:
citrusP = dataframe_to_percentiles(minority_samples)
citrusP = 'P' + citrusP.round().astype(str,errors='ignore')
citrusP['name'] = 0

In [22]:
citrusP

,name,diameter,weight,red,green,blue
6501,0,P31.0,P31.0,P61.0,P71.0,P0.0
7586,0,P53.0,P53.0,P10.0,P23.0,P89.0
7653,0,P55.0,P55.0,P68.0,P12.0,P23.0
6055,0,P22.0,P22.0,P56.0,P67.0,P77.0
5705,0,P16.0,P16.0,P68.0,P88.0,P29.0
...,...,...,...,...,...,...
9111,0,P83.0,P83.0,P53.0,P71.0,P67.0
6902,0,P40.0,P40.0,P15.0,P59.0,P67.0
6703,0,P36.0,P36.0,P24.0,P21.0,P52.0
5121,0,P3.0,P3.0,P27.0,P54.0,P70.0


In [23]:
ZERO = pd.concat([citrusP,random_values], ignore_index=True)
ZERO['name']=0
ZERO

,name,diameter,weight,red,green,blue
0,0,P31.0,P31.0,P61.0,P71.0,P0.0
1,0,P53.0,P53.0,P10.0,P23.0,P89.0
2,0,P55.0,P55.0,P68.0,P12.0,P23.0
3,0,P22.0,P22.0,P56.0,P67.0,P77.0
4,0,P16.0,P16.0,P68.0,P88.0,P29.0
...,...,...,...,...,...,...
4014,0,NaN,P61.0,NaN,NaN,NaN
4015,0,NaN,P80.0,NaN,NaN,NaN
4016,0,NaN,P19.0,NaN,NaN,NaN
4017,0,NaN,P17.0,NaN,NaN,NaN


In [24]:


# Step 1: Identify unique values of 'pH'
unique_size = citrusP['weight'].unique()

# Step 2: Create a dictionary to store non-missing values for each variable
# Initialize the dictionary
imputation_info = {}

# Iterate over each variable (excluding 'citric acid') that has missing values
for column in citrusP.columns:
    if column != 'weight' and ZERO[column].isna().sum() > 0:
        imputation_info[column] = {}
        
        # Iterate over each unique value of 'citric acid'
        for size_value in unique_size:
            # Get the non-missing values of the variable where 'citric acid' is equal to the current size_value
            non_missing_values = citrusP.loc[citrusP['weight'] == size_value, column].dropna().values
            imputation_info[column][size_value] = non_missing_values

In [25]:
# import random
import statistics
def fill_missing_values2(row, imputation_info):
    # For each column, check if it has a missing value
    for column in imputation_info:
        if pd.isna(row[column]):
            size_value = row['weight']  # Get the corresponding pH value for the row
            if size_value in imputation_info[column]:
                possible_values = imputation_info[column][size_value]
                if len(possible_values) > 0:
                    row[column ] = statistics.mode(possible_values)
                    # row[column] = random.choice(possible_values)

    return row

myMode = ZERO.apply(lambda row: fill_missing_values2(row, imputation_info), axis=1)

In [26]:
myMode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4019 entries, 0 to 4018
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      4019 non-null   int64 
 1   diameter  4019 non-null   object
 2   weight    4019 non-null   object
 3   red       4019 non-null   object
 4   green     4019 non-null   object
 5   blue      4019 non-null   object
dtypes: int64(1), object(5)
memory usage: 188.5+ KB


In [27]:
print(myMode.shape)
print(ZERO.shape)

(4019, 6)
(4019, 6)


In [28]:
percentiles = [*range(0,101, 1)]
mapping_data = {}

columns_to_impute = citrus.columns.difference(['name'])
for variable in columns_to_impute:
    mapping_data[variable] = [citrus[citrus['name']==0][variable].quantile(p / 100) for p in percentiles]

# Create the mapping DataFrame
mapping_df = pd.DataFrame(mapping_data, index=[f'P{p}' for p in percentiles])
print("Mapping DataFrame:")
mapping_df

Mapping DataFrame:


,blue,diameter,green,red,weight
P0,2.0,7.6300,31.0,115.0,126.7900
P1,2.0,8.7900,46.0,127.0,152.5894
P2,2.0,9.0600,49.0,130.0,157.5696
P3,2.0,9.1900,51.0,132.0,160.7591
P4,2.0,9.3300,52.0,133.0,163.1276
...,...,...,...,...,...
P96,33.0,13.6104,87.0,168.0,230.7312
P97,34.0,13.7900,89.0,170.0,233.6127
P98,36.0,13.9900,91.0,172.0,236.9422
P99,38.0,14.3200,94.0,174.0,241.5519


In [29]:
def convert_percentiles_to_values(mdf, mapping_df):
    columns_to_impute = mdf.columns.difference(['name'])

    for column in columns_to_impute:
        mdf[column] = mdf[column].apply(lambda x: mapping_df.loc[x, column] if isinstance(x, str) and x.startswith('P') else x)
    return mdf

In [30]:
def path_to_revert(now, then):
    now = now.where(then.isna(), then)
    now = now.replace(r'^(P\d+)\.0$', r'\1', regex=True)
    return now

In [31]:
a = path_to_revert(myMode, ZERO)
a = convert_percentiles_to_values(a, mapping_df)

In [32]:
a

,name,diameter,weight,red,green,blue
0,0,10.86,187.9476,154.0,75.0,2.0
1,0,11.54,198.8600,138.0,63.0,28.0
2,0,11.62,199.8500,155.0,58.0,8.0
3,0,10.54,182.3934,152.0,74.0,23.0
4,0,10.25,178.0568,155.0,82.0,10.0
...,...,...,...,...,...,...
4014,0,11.81,202.9500,142.0,63.0,38.0
4015,0,12.53,213.4760,163.0,66.0,18.0
4016,0,10.41,180.2062,170.0,60.0,9.0
4017,0,10.32,178.9300,143.0,68.0,13.0


In [33]:
percent = pd.concat([a, df_resampled2[df_resampled2['name']==1][0:4019]], ignore_index=True)

In [34]:
percent['name'].value_counts()

name
0    4019
1    4019
Name: count, dtype: int64

In [35]:
X_percent = percent.drop(columns=['name'])  # Replace with actual target column name
y_percent = percent['name']  # Replace with actual target column name

# Standardize the 'percent' data as well
X_percent = scaler.fit_transform(X_percent)

def evaluate_custom_oversampling(X_train, y_train, X_test, y_test, classifiers):
    custom_results = []
    
    # Iterate over classifiers first
    for clf_name, clf in classifiers.items():
        # Train the classifier
        clf.fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = clf.predict(X_test)
        y_prob = clf.predict_proba(X_test)
        
        # Get metrics
        accuracy, precision, recall, f1, auc = get_metrics(y_test, y_pred, y_prob)
        
        # Append the result
        custom_results.append({
            'Classifier': clf_name,               # Classifier first
            'Oversampler': 'PERCENTILES',              # Oversampler (Custom in this case)
            'Accuracy': accuracy,
            'Precision(0)': precision[0],
            'Precision(1)': precision[1],
            'Recall (0)': recall[0],
            'Recall (1)': recall[1],
            'F1(0)': f1[0],
            'F1(1)': f1[1],
            'AUC(0)': auc,
            'AUC(1)': auc
        })
    
    # Convert to DataFrame and return
    return pd.DataFrame(custom_results)



# Evaluate on custom oversampled data (the 'percent' dataset)
custom_results_df = evaluate_custom_oversampling(X_percent, y_percent, X_test, y_test, classifiers)


# Output: Print or review the custom oversampling results
custom_results_df

,Classifier,Oversampler,Accuracy,Precision(0),Precision(1),Recall (0),Recall (1),F1(0),F1(1),AUC(0),AUC(1)
0,GaussianNB,PERCENTILES,0.806716,0.582237,0.993169,0.986072,0.741081,0.732161,0.848803,0.977290,0.977290
1,KNN,PERCENTILES,0.852239,0.651601,0.983931,0.963788,0.811417,0.777528,0.889385,0.944334,0.944334
2,RandomForest,PERCENTILES,0.873881,0.681992,0.996333,0.991643,0.830785,0.808173,0.906059,0.973430,0.973430


In [ ]:
combined_df = pd.concat([results_df, custom_results_df], ignore_index=True)

# Save to Excel
combined_df.to_excel("citrus3.xlsx", index=False)
